In [0]:
options('box.path'='/Workspace/Users/joe.wang@gatesfoundation.org/DHS-Indicators-R')
library(arrow)
box::use(dplyr[...], sparklyr[...], etl/io, etl/metadata, etl/pipeline)
sc <- spark_connect(method = "databricks")

In [0]:
box::use(Chap07_FP/chapter)

In [0]:
pipeline$run(sc, chapter, "7", c("ET"))

In [0]:
ind <- collect(io$read_indicator(sc, "7", c("ET")))
md <- collect(io$read_indicator_metadata(sc, "7"))

ind <- metadata$apply_metadata(ind, md)

In [0]:
ind

In [0]:
box::use(labelled)

In [0]:
labelled::get_value_labels(ind)

In [0]:
#box::reload(chapter)
#box::reload(io)
#box::reload(metadata)